# 02. Building a network from scratch

In this notebook, we will see how to build a little electrical network from scratch, using the PyPI library. Please note that for all **creating** methods used, you can either provide a **full pandas dataframe** or some **lists of values** for each argument/column.

### Importing libraries

In [32]:
import pypowsybl as pp 
import pypowsybl.loadflow as lf
import pandas as pd

### 1. Creating substation

In order to build a network, we will begin by creating the substations we want in our network. Let's suppose we want to create a microgrid focusing on a tie-line between two France and Italie.

In [25]:
n = pp.network.create_empty() # create an empty network
stations = pd.DataFrame.from_records(index='id', data=[
    {'id': 'S1', 'name': 'S1', 'TSO': 'RTE', 'country': 'FR'},
    {'id': 'S2', 'name': 'S2', 'TSO': 'Terna', 'country': 'IT'},
]) # the substations data we want
n.create_substations(stations) # updating the substations in the network
n.get_substations() # getting the substations in the network to check if it worked

,name,TSO,geo_tags,country
id,,,,
S1,S1,RTE,,FR
S2,S2,Terna,,IT


### 2. Creating voltage levels...

Then, let’s add some voltage levels inside those substations, this time with a dataframe:

In [26]:
voltage_levels = pd.DataFrame.from_records(index='id', data=[
    {'substation_id': 'S1', 'id': 'VL1', 'topology_kind': 'BUS_BREAKER', 'nominal_v': 400,'high_voltage_limit': 420, 'low_voltage_limit': 380},
    {'substation_id': 'S2', 'id': 'VL2', 'topology_kind': 'BUS_BREAKER', 'nominal_v': 400,'high_voltage_limit': 420, 'low_voltage_limit': 380},
]) # the voltage levels data we want
n.create_voltage_levels(voltage_levels) # updating the voltage levels in the network	
n.get_voltage_levels() # getting the voltage levels in the network to check if it worked

,name,substation_id,nominal_v,high_voltage_limit,low_voltage_limit
id,,,,,
VL1,,S1,400.0,420.0,380.0
VL2,,S2,400.0,420.0,380.0


### 3. Creating buses & lines

Now, we will create the buses of our network. We will create a bus for each voltage level we have created before.

In [ ]:
buses = pd.DataFrame.from_records(index='id', data=[
    {'voltage_level_id': 'VL1', 'id': 'B1',},
    {'voltage_level_id': 'VL2', 'id': 'B2',},
]) # the buses data we want

n.create_buses(buses) # updating the buses in the network

,name,v_mag,v_angle,connected_component,synchronous_component,voltage_level_id
id,,,,,,


In [28]:
n.create_lines(id='LINE', voltage_level1_id='VL1', bus1_id='B1',
                     voltage_level2_id='VL2', bus2_id='B2',
                     b1=0, b2=0, g1=0, g2=0, r=0.5, x=10)

In [ ]:
n.get_buses() # getting the buses in the network to check if it worked

,name,v_mag,v_angle,connected_component,synchronous_component,voltage_level_id
id,,,,,,
VL1_0,,NaN,NaN,0,0,VL1
VL2_0,,NaN,NaN,0,0,VL2


In [30]:
n.get_lines() # getting the lines in the network to check if it worked

,name,r,x,g1,b1,g2,b2,p1,q1,i1,p2,q2,i2,voltage_level1_id,voltage_level2_id,bus1_id,bus2_id,connected1,connected2
id,,,,,,,,,,,,,,,,,,,
LINE,,0.5,10.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,VL1,VL2,VL1_0,VL2_0,True,True


### Creating load and generators

Now, we will create the loads and generators of our network. We will create a load and a generator for each bus we have created before.

In [ ]:
n.create_loads(id='LOAD', voltage_level_id='VL2', bus_id='B2', p0=100, q0=10)
n.create_generators(id='GEN', voltage_level_id='VL1', bus_id='B1',
                          min_p=0, max_p=200, target_p=100,
                          voltage_regulator_on=True, target_v=400)  


In [ ]:
res = lf.run_ac(n) # running the load flow
str(res[0].status)